# Sample

In [23]:
import stock 
import plotly.graph_objs as go
import plotly.subplots as sp
import pandas as pd 
import vnstock 

df = stock.get_daily_OHLC(from_date='18/01/2022', to_date='23/01/2024', page_size=1000)

df['TradingDate'] = pd.to_datetime(df['TradingDate'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')
df


,Symbol,Market,TradingDate,Time,Open,High,Low,Close,Volume,Value
0,SSI,HOSE,2022/01/18,None,38504,39199,36635,36766,15992300,699221410000.0020
1,SSI,HOSE,2022/01/19,None,38156,39286,37678,39286,11085500,488587305000.0080
2,SSI,HOSE,2022/01/20,None,39460,40155,38634,39721,9055600,409030700000.0030
3,SSI,HOSE,2022/01/21,None,39286,39634,38417,38721,8527900,383190785000.0010
4,SSI,HOSE,2022/01/24,None,38069,38504,36027,36027,18790500,793994135000.0370
...,...,...,...,...,...,...,...,...,...,...
499,SSI,HOSE,2024/01/17,None,33500,34500,33450,33650,26218900,892270305000.0280
500,SSI,HOSE,2024/01/18,None,33800,34050,33650,33800,14736700,499010570000.0080
501,SSI,HOSE,2024/01/19,None,34000,34100,33400,33700,22389300,754091994999.97
502,SSI,HOSE,2024/01/22,None,33900,34000,33400,33950,16814700,565471185000.0010


In [40]:
def candlestick_chart(df, title='Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', ma_periods=None, show_volume=True, figure_size=(15, 8), reference_period=None, colors=('#00F4B0', '#FF3747'), reference_colors=('blue', 'black')):
    """
    Generate a candlestick chart with optional Moving Averages (MA) lines, volume data, and reference lines.

    Parameters:
    - df: DataFrame with candlestick data ('time', 'open', 'high', 'low', 'close', 'volume', 'ticker').
    - title: Title of the chart.
    - x_label: Label for the x-axis.
    - y_label: Label for the y-axis.
    - ma_periods: List of MA periods to calculate and plot (e.g., [10, 50, 200]).
    - show_volume: Boolean to indicate whether to display volume data.
    - figure_size: Tuple specifying the figure size (width, height).
    - reference_period: Number of days to consider for reference lines (e.g., 90).
    - colors: Tuple of color codes for up and down candles (e.g., ('#00F4B0', '#FF3747')).
    - reference_colors: Tuple of color codes for reference lines (e.g., ('black', 'blue')).

    Returns:
    - Plotly figure object.
    """
    # Create the base candlestick chart
    candlestick_trace = go.Candlestick(
        x=df['TradingDate'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick',
    )

    # Create a figure
    fig = go.Figure(data=[candlestick_trace])

    # Add volume data if specified
    if show_volume:
        df['Volume']=df['Volume'].astype(float)
        volume_trace = go.Bar(
            x=df['TradingDate'],
            y=df['Volume'],
            name='Volume',
            yaxis='y2',  # Use the secondary y-axis for volume
            marker=dict(color=[colors[0] if close >= open else colors[1] for close, open in zip(df['Close'], df['Open'])]),  # Match volume color to candle color
        )

        fig.add_trace(volume_trace)

    # Add Moving Averages (MA) lines if specified
    if ma_periods:
        for period in ma_periods:
            ma_name = f'{period}-day MA'
            df[ma_name] = df['Close'].rolling(period).mean()

            ma_trace = go.Scatter(
                x=df['TradingDate'],
                y=df[ma_name],
                mode='lines',
                name=ma_name,
            )

            fig.add_trace(ma_trace)

    # Add straight reference lines for the highest high and lowest low
    if reference_period:
        df['lowest_low'] = df['Low'].rolling(reference_period).min()
        df['highest_high'] = df['High'].rolling(reference_period).max()

        lowest_low_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['lowest_low'].iloc[-1]] * len(df),  # Create a straight line for lowest low
            mode='lines',
            name=f'Lowest Low ({reference_period} days)',
            line=dict(color=reference_colors[0], dash='dot'),
        )

        highest_high_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['highest_high'].iloc[-1]] * len(df),  # Create a straight line for highest high
            mode='lines',
            name=f'Highest High ({reference_period} days)',
            line=dict(color=reference_colors[1], dash='dot'),
        )

        fig.add_trace(lowest_low_trace)
        fig.add_trace(highest_high_trace)

    # Customize the chart appearance
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis_rangeslider_visible=True,
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right',
        ),
        width=figure_size[0] * 100,  # Convert short form to a larger size for better readability
        height=figure_size[1] * 100,
        margin=dict(l=50, r=50, t=70, b=50),  # Adjust margins for space between title and legend
    )

    return fig

In [50]:
candlestick_chart(df, ma_periods=[5], show_volume= True, reference_period=200)

In [14]:
candlestick_trace = go.Candlestick(
    x=df['TradingDate'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    
)
# Create a figure
fig = go.Figure(data=[candlestick_trace])
fig

In [73]:
stock.get_securities_list()

,Market,Symbol,StockName,StockEnName
0,HOSE,AAA,CTCP NHUA AN PHAT XANH,An Phat Bioplastics Joint Stock Company
1,HOSE,AAM,CTCP THUY SAN MEKONG,Mekong Fisheries Joint Stock Company
2,HOSE,AAT,CTCP TD TIEN SON TH,Tien Son Thanh Hoa Group Joint Stock Company
3,HOSE,ABR,CTCP DT NHAN HIEU VIET,Viet Brand Invest Joint Stock Company
4,HOSE,ABS,CTCP DV NN BINH THUAN,BinhThuan Agriculture Services Joint Stock Com...
...,...,...,...,...
95,HOSE,CHPG2340,CQ CHPG2340,None
96,HOSE,CHPG2341,CQ CHPG2341,None
97,HOSE,CHPG2342,CQ CHPG2342,None
98,HOSE,CHPG2343,CQ CHPG2343,None


In [4]:
colors=('#00F4B0', '#FF3747')
candlestick_trace = go.Candlestick(
    x=df['TradingDate'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Candlestick',
)

df['Volume']=df['Volume'].astype(float)
volume_trace = go.Bar(
    x=df['TradingDate'],
    y=df['Volume'],
    name='Volume',
    yaxis='y2',  # Use the secondary y-axis for volume
    marker=dict(color=[colors[0] if close >= open else colors[1] for close, open in zip(df['Close'], df['Open'])]),  # Match volume color to candle color
)

fig = sp.make_subplots(rows=2, cols=1, shared_xaxes= True, vertical_spacing=0.02)

# Add candlestick chart to the first subplot
fig.add_trace(candlestick_trace, row=1, col=1)

# Add volume chart to the second subplot
fig.add_trace(volume_trace, row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False,
                  showlegend=False,
                  hovermode="x",
                xaxis=dict(
                showspikes=True,
                spikethickness=2,
                spikedash="dot",
                spikecolor="rgba(0, 0, 0, 0.5)",
                spikemode="across",),
                )  


fig

In [2]:
import requests
url = 'https://fiin-fundamental.ssi.com.vn/FinancialStatement/DownloadBalanceSheet?language=vi&OrganCode=SSI&Skip=0&Frequency=Quarterly'
response = requests.get(url)
response.status_code

403

In [7]:
import stock
stock.get_intraday_OHLC(page_size=250) 

,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,SSI,18300,15/10/2020,09:15:56,18300,18300,18300,18300,244170
1,SSI,18300,15/10/2020,09:16:35,18300,18350,18250,18300,42440
2,SSI,18350,15/10/2020,09:17:54,18300,18350,18300,18350,10470
3,SSI,18300,15/10/2020,09:18:59,18350,18350,18300,18300,74870
4,SSI,18300,15/10/2020,09:19:48,18300,18300,18300,18300,31430
...,...,...,...,...,...,...,...,...,...
213,SSI,18150,15/10/2020,14:26:58,18100,18150,18100,18150,25700
214,SSI,18150,15/10/2020,14:27:58,18150,18150,18100,18150,5180
215,SSI,18150,15/10/2020,14:28:55,18150,18150,18150,18150,20070
216,SSI,18150,15/10/2020,14:29:46,18150,18150,18150,18150,16000


In [20]:
tickers = stock.get_securities_list(page_size=1000)['Symbol']
tickers_list = ",".join(tickers.tolist())
tickers_list


'AAA,AAM,AAT,ABR,ABS,ABT,ACB,ACC,ACG,ACL,ADG,ADP,ADS,AGG,AGM,AGR,ANV,APC,APG,APH,ASG,ASM,ASP,AST,BAF,BBC,BCE,BCG,BCM,BFC,BHN,BIC,BID,BKG,BMC,BMI,BMP,BRC,BSI,BTP,BTT,BVH,BWE,C32,C47,CACB2303,CACB2304,CACB2305,CACB2306,CACB2307,CAV,CCI,CCL,CDC,CFPT2305,CFPT2307,CFPT2309,CFPT2310,CFPT2311,CFPT2312,CFPT2313,CFPT2314,CFPT2315,CFPT2316,CFPT2317,CFPT2318,CHDB2304,CHDB2305,CHDB2306,CHP,CHPG2309,CHPG2313,CHPG2314,CHPG2315,CHPG2316,CHPG2317,CHPG2318,CHPG2319,CHPG2321,CHPG2322,CHPG2325,CHPG2326,CHPG2328,CHPG2329,CHPG2330,CHPG2331,CHPG2332,CHPG2333,CHPG2334,CHPG2335,CHPG2336,CHPG2337,CHPG2338,CHPG2339,CHPG2340,CHPG2341,CHPG2342,CHPG2343,CIG,CII,CKG,CLC,CLL,CLW,CMBB2306,CMBB2308,CMBB2309,CMBB2311,CMBB2312,CMBB2313,CMBB2314,CMBB2315,CMBB2316,CMBB2317,CMBB2318,CMG,CMSN2302,CMSN2305,CMSN2306,CMSN2307,CMSN2309,CMSN2311,CMSN2312,CMSN2313,CMSN2314,CMSN2315,CMSN2316,CMSN2317,CMV,CMWG2305,CMWG2307,CMWG2309,CMWG2310,CMWG2311,CMWG2312,CMWG2313,CMWG2314,CMWG2315,CMWG2316,CMWG2317,CMWG2318,CMX,CNG,CNVL2303,CNV

In [21]:
vnstock.price_depth(tickers_list)

,Mã CP,Giá tham chiếu,Giá Trần,Giá Sàn,Giá mua 3,KL mua 3,Giá mua 2,KL mua 2,Giá mua 1,KL mua 1,...,Giá bán 1,KL bán 1,Giá bán 2,KL bán 2,Giá bán 3,KL bán 3,Tổng Khối Lượng,ĐTNN Mua,ĐTNN Bán,ĐTNN Room
0,AAA,10100,10800,9400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,373544183
1,AAM,9060,9690,8430,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5942764
2,AAT,5410,5780,5040,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34798951
3,ABR,13300,14200,12400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,69195
4,ABS,5990,6400,5580,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,VSI,18100,19350,16850,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6343540
624,VTB,11200,11950,10450,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5646615
625,VTO,9850,10500,9170,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35581553
626,YBM,8600,9200,8000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6968595


In [2]:
import stock
stock.get_financial_ratio('SSI', 'yearly')

year,2022,2021,2020,2019,2018
ticker,SSI,SSI,SSI,SSI,SSI
priceToEarning,15.0,25.0,23.8,10.6,10.5
priceToBook,1.1,4.8,3.1,1.0,1.5
valueBeforeEbitda,15.6,16.9,15.5,11.0,12.6
dividend,0.059,0.022,0.05,0.102,0.1
roe,0.093,0.225,0.131,0.099,0.148
roa,0.033,0.062,0.04,0.036,0.061
daysReceivable,835.0,326.0,380.0,NaN,NaN
daysPayable,23,3,6,6,2
earningPerShare,1133,1798,844,925,1328
